# _Book Rental - Part 2_ 
## _Building the Recommendation Model_
***

In [17]:
#library imports
import pandas as pd
import numpy as np
#handle the warnings
import warnings
warnings.filterwarnings(action='ignore',category=DeprecationWarning)
warnings.filterwarnings(action='ignore',category=FutureWarning)

In [18]:
#load the data generated from Part 1
data =  pd.read_csv('recommender.csv',delimiter=',',engine='python')
data.head()

,user_id,isbn,rating,book_title,book_author,year_of_publication,publisher,num_user_id,num_isbn
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,0,0
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,1,0
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,2,0
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,3,0
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,4,0


In [20]:
#rearrange the columns in --> num_user_id, num_isbn and rating
data_matrix = data[['num_user_id','num_isbn','rating']]
data_matrix.head()

,num_user_id,num_isbn,rating
0,0,0,0
1,1,0,5
2,2,0,0
3,3,0,5
4,4,0,9


In [21]:
data_matrix['rating'].unique()

array([ 0,  5,  9, 10,  3,  6,  7,  8,  4,  1,  2])

In [22]:
data_matrix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43988 entries, 0 to 43987
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   num_user_id  43988 non-null  int64
 1   num_isbn     43988 non-null  int64
 2   rating       43988 non-null  int64
dtypes: int64(3)
memory usage: 1.0 MB


## _Modeling - surprise-sklearn_

In [43]:
#import the libraries
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
#model and metrics
from surprise import KNNWithMeans
from surprise import accuracy
#default dict to store all the predictions as dict which could vary
from collections import defaultdict

In [37]:
#read the data from the dataframe created
reader = Reader(rating_scale=(0,10))
matrix = Dataset.load_from_df(data_matrix,reader)
#instantiate a model
model = KNNWithMeans(k=50,min_k=1,sim_options={'name':'cosine','user_based':True})

In [38]:
#split the data matrix into train and test set
train_set,test_set = train_test_split(matrix,test_size=0.20,random_state=42)

In [39]:
#train the model on the train set
model.fit(train_set)

Computing the cosine similarity matrix...


/usr/local/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.


In [40]:
#predict on the test set 
test_predict = model.test(test_set)

In [44]:
#get the performance score
print('User based Recommendation System Model - RMSE on Test Set')
accuracy.rmse(test_predict,verbose=True)

User based Recommendation System Model - RMSE on Test Set
RMSE: 4.0282


4.028248934448262

In [45]:
#get the top n predictions
def top_n_predictions(predictions,n=10):
    '''Returns the top n predictions for each of the user based on the predictions'''
    #first map predictions to each user 
    top_n = defaultdict(list) #https://www.geeksforgeeks.org/defaultdict-in-python/
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n
    
    

In [54]:
top_n = top_n_predictions(test_predict, n=10)
# Print the recommended items for each user
counter = 0
for uid, user_ratings in top_n.items():
    if counter == 10:
        break
    else:
        print(uid, [iid for (iid, _) in user_ratings])
        counter = counter + 1    

313 [1204, 9745, 18441, 18419, 18356, 18420, 18367, 18412, 18353, 18413]
1178 [1824, 19104, 19101]
4031 [24076]
206 [8827, 10329, 5863, 10380, 10406, 10347, 10408, 10413, 10429, 10424]
20 [2764, 13518, 8572, 19776, 16517, 25226, 19138, 9130, 17716, 7493]
7 [2288, 109, 937, 3682, 5025, 1954, 4085, 5014, 2571, 2923]
69 [8225, 1098, 2652, 8197, 8240, 8106, 8309, 8208, 8260, 8295]
2007 [7416]
1914 [8460, 8451, 7180]
900 [1365, 25690, 25668, 25666, 25635, 22870, 25667, 25633, 25684, 25687]
